In [4]:
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import os
import itertools
import tensorflow


In [3]:
batch_size = 128 # batch 的大小，如果出現 OOM error，請降低這個值
num_classes = 10 # 類別的數量，Cifar 10 共有 10 個類別
epochs = 10 # 訓練的 epochs 數量

# 讀取資料並檢視
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# 對 label 進行 one-hot encoding (y_trian 原本是純數字)
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


## 首先我們使用一般的 DNN (MLP) 來訓練
由於 DNN 只能輸入一維的資料，我們要先將影像進行攤平，若 (50000, 32, 32, 3) 的影像，攤平後會變成 (50000, 32*32*3) = (50000, 3072)

In [4]:
# 將資料攤平成一維資料
x_train = x_train.reshape(50000, 3072) 
x_test = x_test.reshape(10000, 3072)

# 將資料變為 float32 並標準化
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

50000 train samples
10000 test samples


In [5]:
model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(3072,)))
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(num_classes, activation='softmax'))

model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 512)               1573376   
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                5130      
Total params: 1,841,162
Trainable params: 1,841,162
Non-trainable params: 0
_________________________________________________________________
Train on 50000 samples, validate on 10000 samples
Epoch 1/10
50000/50000 [==============================] - 5s 93us/step - loss: 2.1

## 接下來我們使用 CNN 來訓練神經網路
CNN 的原理非常適合處理影像類的資料，就讓我們來看看，同樣的訓練條件，CNN 是否顯著優於 DNN 呢?

In [6]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [7]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))
model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 30, 30, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 30, 30, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 15, 15, 64)        18496     
__________

## 同樣運算 10 個 epochs，但 CNN 在 test data 的準確率顯著優於 DNN!

## 作業
1. 請試著調整各個超參數，並說明那些超參數對於結果有明顯的影響?
2. CNN 與 DNN 哪個模型的參數數量比較多? 造成參數的數量不同的原因在哪?

In [53]:
LEARNING_RATE = 1e-3
BATCH_SIZE = 512 
NUM_CLASSES = 10 # 類別的數量，Cifar 10 共有 10 個類別


In [54]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

# Convert class vectors to binary class matrices.
y_train = keras.utils.np_utils.to_categorical(y_train, NUM_CLASSES)
y_test = keras.utils.np_utils.to_categorical(y_test, NUM_CLASSES)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [77]:
optimizer_set = [keras.optimizers.gradient_descent_v2.SGD(learning_rate=LEARNING_RATE, nesterov=True, momentum=0.95),
                 tensorflow.keras.optimizers.Adam(learning_rate=LEARNING_RATE),
                 tensorflow.keras.optimizers.RMSprop(learning_rate=LEARNING_RATE)]

"""Code Here
建立實驗的比較組合
"""
epochs = [10, 20]
kernels = [(3,3), (5, 5)]
pool_sizes = [(2,2), (3, 3)]


In [78]:

results = {}
for i, (optim, kernel, epochs, pool_size) in enumerate(itertools.product(optimizer_set, kernels, epochs, pool_sizes )):
    print(f"Numbers of exp: {i}, optimizer:{type(optim).__name__}, kernel: {kernel}, epochs: {epochs}, pool_size:{pool_size}")

    model = Sequential()
    model.add(Conv2D(32, kernel, padding='same',
                     input_shape=x_train.shape[1:]))
    model.add(Activation('relu'))
    model.add(Conv2D(32, kernel))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=pool_size))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, kernel, padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(64, kernel))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=pool_size))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(NUM_CLASSES))
    model.add(Activation('softmax'))
    model.summary()
    
    
    model.compile(loss='categorical_crossentropy',
                  optimizer=optim,
                  metrics=['accuracy'])

    history = model.fit(x_train, y_train,
                        batch_size=BATCH_SIZE,
                        epochs=epochs,
                        verbose=1,
                        validation_data=(x_test, y_test))
    score = model.evaluate(x_test, y_test, verbose=0)
    
    test_loss = score[0]
    
    test_accuracy = score[1]
    
    # Collect results
    print(f"test_loss: {test_loss}, test_accuracy: {test_accuracy}")
    exp_name_tag = (f"Numbers of exp: {i}, optimizer:{type(optim).__name__}, kernel: {kernel}, epochs: {epochs}, pool_size:{pool_size}")
    results[exp_name_tag] = {#'train-loss': model.history.history["loss"],
                             #'valid-loss': model.history.history["val_loss"],
                             #'train-acc': model.history.history["accuracy"],
                             #'valid-acc': model.history.history["val_accuracy"],
                             'Test loss': test_loss,
                             'test accuracy': test_accuracy
                            }
    

Numbers of exp: 0, optimizer:SGD, kernel: (3, 3), epochs: 10, pool_size:(2, 2)
Model: "sequential_45"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_180 (Conv2D)          (None, 32, 32, 32)        896       
_________________________________________________________________
activation_265 (Activation)  (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_181 (Conv2D)          (None, 30, 30, 32)        9248      
_________________________________________________________________
activation_266 (Activation)  (None, 30, 30, 32)        0         
_________________________________________________________________
max_pooling2d_90 (MaxPooling (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_131 (Dropout)        (None, 15, 15, 32)        0         
________________________________________

98/98 [==============================] - 53s 537ms/step - loss: 2.0556 - accuracy: 0.2184 - val_loss: 2.0039 - val_accuracy: 0.2563
Epoch 9/10
98/98 [==============================] - 53s 539ms/step - loss: 2.0353 - accuracy: 0.2249 - val_loss: 1.9829 - val_accuracy: 0.2570
Epoch 10/10
98/98 [==============================] - 52s 534ms/step - loss: 2.0147 - accuracy: 0.2330 - val_loss: 1.9613 - val_accuracy: 0.2652
test_loss: 1.9612817764282227, test_accuracy: 0.2651999890804291
Numbers of exp: 2, optimizer:SGD, kernel: (3, 3), epochs: 20, pool_size:(2, 2)
Model: "sequential_47"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_188 (Conv2D)          (None, 32, 32, 32)        896       
_________________________________________________________________
activation_277 (Activation)  (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_189 (Conv2D

Epoch 1/20
98/98 [==============================] - 53s 542ms/step - loss: 2.3038 - accuracy: 0.1029 - val_loss: 2.2997 - val_accuracy: 0.1434
Epoch 2/20
98/98 [==============================] - 53s 536ms/step - loss: 2.2996 - accuracy: 0.1129 - val_loss: 2.2969 - val_accuracy: 0.1865
Epoch 3/20
98/98 [==============================] - 53s 539ms/step - loss: 2.2956 - accuracy: 0.1231 - val_loss: 2.2924 - val_accuracy: 0.1891
Epoch 4/20
98/98 [==============================] - 52s 534ms/step - loss: 2.2881 - accuracy: 0.1350 - val_loss: 2.2804 - val_accuracy: 0.1707
Epoch 5/20
98/98 [==============================] - 52s 533ms/step - loss: 2.2683 - accuracy: 0.1536 - val_loss: 2.2447 - val_accuracy: 0.1888
Epoch 6/20
98/98 [==============================] - 53s 537ms/step - loss: 2.2133 - accuracy: 0.1704 - val_loss: 2.1512 - val_accuracy: 0.2344
Epoch 7/20
98/98 [==============================] - 53s 537ms/step - loss: 2.1308 - accuracy: 0.1920 - val_loss: 2.0664 - val_accuracy: 0.2427

Epoch 1/10
98/98 [==============================] - 122s 1s/step - loss: 2.3016 - accuracy: 0.1042 - val_loss: 2.2977 - val_accuracy: 0.1103
Epoch 2/10
98/98 [==============================] - 134s 1s/step - loss: 2.2936 - accuracy: 0.1165 - val_loss: 2.2843 - val_accuracy: 0.1278
Epoch 3/10
98/98 [==============================] - 127s 1s/step - loss: 2.2660 - accuracy: 0.1377 - val_loss: 2.2198 - val_accuracy: 0.1931
Epoch 4/10
98/98 [==============================] - 135s 1s/step - loss: 2.1808 - accuracy: 0.1783 - val_loss: 2.1088 - val_accuracy: 0.2186
Epoch 5/10
98/98 [==============================] - 130s 1s/step - loss: 2.1153 - accuracy: 0.1962 - val_loss: 2.0583 - val_accuracy: 0.2242
Epoch 6/10
98/98 [==============================] - 119s 1s/step - loss: 2.0827 - accuracy: 0.2084 - val_loss: 2.0327 - val_accuracy: 0.2523
Epoch 7/10
98/98 [==============================] - 112s 1s/step - loss: 2.0514 - accuracy: 0.2216 - val_loss: 1.9922 - val_accuracy: 0.2666
Epoch 8/10
98

Epoch 1/20
98/98 [==============================] - 101s 1s/step - loss: 2.3016 - accuracy: 0.1037 - val_loss: 2.2996 - val_accuracy: 0.1248
Epoch 2/20
98/98 [==============================] - 100s 1s/step - loss: 2.2979 - accuracy: 0.1184 - val_loss: 2.2933 - val_accuracy: 0.1342
Epoch 3/20
98/98 [==============================] - 101s 1s/step - loss: 2.2856 - accuracy: 0.1318 - val_loss: 2.2644 - val_accuracy: 0.1739
Epoch 4/20
98/98 [==============================] - 100s 1s/step - loss: 2.2276 - accuracy: 0.1610 - val_loss: 2.1484 - val_accuracy: 0.1935
Epoch 5/20
98/98 [==============================] - 101s 1s/step - loss: 2.1376 - accuracy: 0.1891 - val_loss: 2.0720 - val_accuracy: 0.2286
Epoch 6/20
98/98 [==============================] - 101s 1s/step - loss: 2.0923 - accuracy: 0.1998 - val_loss: 2.0336 - val_accuracy: 0.2519
Epoch 7/20
98/98 [==============================] - 101s 1s/step - loss: 2.0549 - accuracy: 0.2132 - val_loss: 1.9883 - val_accuracy: 0.2698
Epoch 8/20
98

...

Epoch 1/20
98/98 [==============================] - 101s 1s/step - loss: 2.1737 - accuracy: 0.1812 - val_loss: 1.9162 - val_accuracy: 0.2870
Epoch 2/20
98/98 [==============================] - 100s 1s/step - loss: 1.9137 - accuracy: 0.2918 - val_loss: 1.7822 - val_accuracy: 0.3504
Epoch 3/20
98/98 [==============================] - 103s 1s/step - loss: 1.8340 - accuracy: 0.3246 - val_loss: 1.7180 - val_accuracy: 0.3739
Epoch 4/20
98/98 [==============================] - 102s 1s/step - loss: 1.7797 - accuracy: 0.3441 - val_loss: 1.6691 - val_accuracy: 0.3943
Epoch 5/20
98/98 [==============================] - 100s 1s/step - loss: 1.7173 - accuracy: 0.3754 - val_loss: 1.6451 - val_accuracy: 0.4060
Epoch 6/20
98/98 [==============================] - 102s 1s/step - loss: 1.6777 - accuracy: 0.3889 - val_loss: 1.5918 - val_accuracy: 0.4248
Epoch 7/20
98/98 [==============================] - 100s 1s/step - loss: 1.6406 - accuracy: 0.4035 - val_loss: 1.5436 - val_accuracy: 0.4483
Epoch 8/20
98

Epoch 1/10
98/98 [==============================] - 52s 522ms/step - loss: 2.1096 - accuracy: 0.2187 - val_loss: 1.8848 - val_accuracy: 0.3359
Epoch 2/10
98/98 [==============================] - 51s 522ms/step - loss: 1.8466 - accuracy: 0.3300 - val_loss: 1.5973 - val_accuracy: 0.4338
Epoch 3/10
98/98 [==============================] - 51s 520ms/step - loss: 1.6668 - accuracy: 0.3959 - val_loss: 1.4544 - val_accuracy: 0.4790
Epoch 4/10
98/98 [==============================] - 51s 521ms/step - loss: 1.5418 - accuracy: 0.4441 - val_loss: 1.5120 - val_accuracy: 0.4620
Epoch 5/10
98/98 [==============================] - 51s 517ms/step - loss: 1.4587 - accuracy: 0.4782 - val_loss: 1.3492 - val_accuracy: 0.5089
Epoch 6/10
98/98 [==============================] - 51s 517ms/step - loss: 1.3840 - accuracy: 0.5062 - val_loss: 1.2515 - val_accuracy: 0.5615
Epoch 7/10
98/98 [==============================] - 51s 519ms/step - loss: 1.3318 - accuracy: 0.5274 - val_loss: 1.2807 - val_accuracy: 0.5514

Epoch 1/20
98/98 [==============================] - 52s 526ms/step - loss: 2.1047 - accuracy: 0.2230 - val_loss: 1.9324 - val_accuracy: 0.2985
Epoch 2/20
98/98 [==============================] - 51s 525ms/step - loss: 1.8396 - accuracy: 0.3349 - val_loss: 1.9506 - val_accuracy: 0.3081
Epoch 3/20
98/98 [==============================] - 51s 520ms/step - loss: 1.6850 - accuracy: 0.3900 - val_loss: 1.4878 - val_accuracy: 0.4657
Epoch 4/20
98/98 [==============================] - 52s 531ms/step - loss: 1.5741 - accuracy: 0.4307 - val_loss: 1.5473 - val_accuracy: 0.4452
Epoch 5/20
98/98 [==============================] - 51s 523ms/step - loss: 1.4885 - accuracy: 0.4660 - val_loss: 1.3333 - val_accuracy: 0.5217
Epoch 6/20
98/98 [==============================] - 51s 523ms/step - loss: 1.4100 - accuracy: 0.4918 - val_loss: 1.2716 - val_accuracy: 0.5456
Epoch 7/20
98/98 [==============================] - 53s 543ms/step - loss: 1.3450 - accuracy: 0.5198 - val_loss: 1.3240 - val_accuracy: 0.5237

Epoch 1/10
98/98 [==============================] - 102s 1s/step - loss: 2.2144 - accuracy: 0.1733 - val_loss: 1.9708 - val_accuracy: 0.3001
Epoch 2/10
98/98 [==============================] - 101s 1s/step - loss: 1.9315 - accuracy: 0.2901 - val_loss: 1.8109 - val_accuracy: 0.3531
Epoch 3/10
98/98 [==============================] - 101s 1s/step - loss: 1.7313 - accuracy: 0.3692 - val_loss: 1.4834 - val_accuracy: 0.4700
Epoch 4/10
98/98 [==============================] - 102s 1s/step - loss: 1.5823 - accuracy: 0.4294 - val_loss: 1.3874 - val_accuracy: 0.4978
Epoch 5/10
98/98 [==============================] - 101s 1s/step - loss: 1.4697 - accuracy: 0.4751 - val_loss: 1.3561 - val_accuracy: 0.5131
Epoch 6/10
98/98 [==============================] - 102s 1s/step - loss: 1.3865 - accuracy: 0.5074 - val_loss: 1.2462 - val_accuracy: 0.5614
Epoch 7/10
98/98 [==============================] - 102s 1s/step - loss: 1.3105 - accuracy: 0.5366 - val_loss: 1.2265 - val_accuracy: 0.5667
Epoch 8/10
98

Epoch 1/20
98/98 [==============================] - 102s 1s/step - loss: 2.1940 - accuracy: 0.1838 - val_loss: 1.9765 - val_accuracy: 0.3165
Epoch 2/20
98/98 [==============================] - 102s 1s/step - loss: 1.9060 - accuracy: 0.3088 - val_loss: 1.6470 - val_accuracy: 0.4008
Epoch 3/20
98/98 [==============================] - 101s 1s/step - loss: 1.6999 - accuracy: 0.3803 - val_loss: 1.5843 - val_accuracy: 0.4277
Epoch 4/20
98/98 [==============================] - 102s 1s/step - loss: 1.5666 - accuracy: 0.4332 - val_loss: 1.6195 - val_accuracy: 0.4186
Epoch 5/20
98/98 [==============================] - 102s 1s/step - loss: 1.4595 - accuracy: 0.4756 - val_loss: 1.5901 - val_accuracy: 0.4550
Epoch 6/20
98/98 [==============================] - 102s 1s/step - loss: 1.3745 - accuracy: 0.5099 - val_loss: 1.2598 - val_accuracy: 0.5450
Epoch 7/20
98/98 [==============================] - 101s 1s/step - loss: 1.2939 - accuracy: 0.5409 - val_loss: 1.3983 - val_accuracy: 0.5204
Epoch 8/20
98

In [80]:
results

{'Numbers of exp: 0, optimizer:SGD, kernel: (3, 3), epochs: 10, pool_size:(2, 2)': {'Test loss': 1.7792236804962158,
  'test accuracy': 0.37880000472068787},
 'Numbers of exp: 1, optimizer:SGD, kernel: (3, 3), epochs: 10, pool_size:(3, 3)': {'Test loss': 1.9612817764282227,
  'test accuracy': 0.2651999890804291},
 'Numbers of exp: 2, optimizer:SGD, kernel: (3, 3), epochs: 20, pool_size:(2, 2)': {'Test loss': 1.417934536933899,
  'test accuracy': 0.4846000075340271},
 'Numbers of exp: 3, optimizer:SGD, kernel: (3, 3), epochs: 20, pool_size:(3, 3)': {'Test loss': 1.6679441928863525,
  'test accuracy': 0.3921000063419342},
 'Numbers of exp: 4, optimizer:SGD, kernel: (5, 5), epochs: 10, pool_size:(2, 2)': {'Test loss': 1.534393310546875,
  'test accuracy': 0.4415999948978424},
 'Numbers of exp: 5, optimizer:SGD, kernel: (5, 5), epochs: 10, pool_size:(3, 3)': {'Test loss': 1.8250279426574707,
  'test accuracy': 0.32109999656677246},
 'Numbers of exp: 6, optimizer:SGD, kernel: (5, 5), epochs

In [127]:
import re
import pandas as pd 
exp = []
for i, cond in enumerate(results.keys()):
    params = re.split(', [a-zA-Z]{1}', cond)
    params_list = [p.split(':')[1].strip() for p in params]
    params_list.append(results[cond]['Test loss'])
    params_list.append(results[cond]['test accuracy'])
    exp.append(params_list)

cols = ['Numbers of exp', 'optimizer', 'kernel', 'epochs', 'pool_size', 'Test loss', 'test accuracy']
experiments = pd.DataFrame(exp, columns =cols)

In [128]:
experiments

,Numbers of exp,optimizer,kernel,epochs,pool_size,Test loss,test accuracy
0,0,SGD,"(3, 3)",10,"(2, 2)",1.779224,0.3788
1,1,SGD,"(3, 3)",10,"(3, 3)",1.961282,0.2652
2,2,SGD,"(3, 3)",20,"(2, 2)",1.417935,0.4846
3,3,SGD,"(3, 3)",20,"(3, 3)",1.667944,0.3921
4,4,SGD,"(5, 5)",10,"(2, 2)",1.534393,0.4416
5,5,SGD,"(5, 5)",10,"(3, 3)",1.825028,0.3211
6,6,SGD,"(5, 5)",20,"(2, 2)",1.315719,0.5229
7,7,SGD,"(5, 5)",20,"(3, 3)",1.486342,0.4531
8,8,Adam,"(3, 3)",10,"(2, 2)",0.759319,0.7374
9,9,Adam,"(3, 3)",10,"(3, 3)",1.000064,0.6442


# 作業
1. 請試著調整各個超參數，並說明那些超參數對於結果有明顯的影響?
    > 上述實驗 Optimizer 與 epoch 有明顯的影響
2. CNN 與 DNN 哪個模型的參數數量比較多? 造成參數的數量不同的原因在哪?
    > DNＮ比較多，因為池化層會減少CNN參數量

In [86]:
results['Numbers of exp: 22, optimizer:RMSprop, kernel: (5, 5), epochs: 20, pool_size:(2, 2)']['Test loss']

0.8670749068260193

In [124]:
import re
params = 'Numbers of exp: 22, optimizer:RMSprop, kernel: (5, 5), epochs: 20, pool_size:(2, 2)'
l = [1, 2, 3]
param = re.split(', [a-zA-Z]{1}', params)
a = [p.split(':')[1].strip() for p in param]
b = a + l
b

['22', 'RMSprop', '(5, 5)', '20', '(2, 2)', 1, 2, 3]